![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)
### ODPi Egeria Hands-On Lab
# Welcome to the Configuring Discovery Servers Lab

## Introduction

ODPi Egeria is an open source project that provides open standards and implementation libraries to connect tools,
catalogues and platforms together so they can share information about data and technology (called metadata).

In this hands-on lab you will get a chance to run an Egeria metadata server, configure discovery services in a discovery engine and run the discovery engine in a discovery server.

## What is open discovery?

Open discovery is the ability to automatically analyse and create metadata about assets.  ODPi Egeria provides an Open Discovery Framework (ODF) that defines open interfaces for components that provide specific types of discovery capability so that they can be called from discovery platforms offered by different vendors.

The Open Discovery Framework (ODF) provides standard interfaces for building **discovery services** and grouping them together into a useful collection of capability called a **discovery engine**.

ODPi Egeria then provides a governance server called the **discovery server** that can host one or more discovery engines.

## The scenario

Gary Geeke is the IT Infrastructure leader at Coco Pharmaceuticals.  He has set up a number of OMAG Server Platforms and
is configuring the servers to run on them.

![Gary Geeke](https://raw.githubusercontent.com/odpi/data-governance/master/docs/coco-pharmaceuticals/personas/gary-geeke.png)

In this hands-on lab Gary is setting up a discovery server for the data lake.  Gary's userId is `garygeeke`.

In [ ]:
import requests

adminUserId      = "garygeeke"
organizationName = "Coco Pharmaceuticals"
serverType       = "Open Discovery Server"

In the **Metadata Server Configuration** lab, Gary configured servers for the OMAG Server Platforms shown in Figure 1:

![Figure 1](../images/coco-pharmaceuticals-systems-omag-server-platforms.png)
> **Figure 1:** Coco Pharmaceuticals' OMAG Server Platforms

The discovery server will run on the Data Lake OMAG Server Platform.

In [ ]:
%run ../common/environment-check.ipynb

----
There are two parts to setting up a discovery server.  First the configuration for a discovery engine needs to be created and added to a metadata server.  In this example, this will be the Data Lake Operations Metadata Server called `cocoMDS1`.

In [ ]:
mdrServerName = "cocoMDS1"

Then the discovery server is configured with the location of the metadata server and the identity of one or more discovery engines.
When the discovery server starts it contacts the metadata server and retrieves the configuration for the discovery engine(s).

## Exercise 1 - Configuring the Discovery Engine

The discovery engine is configured using calls to the Discovery Engine OMAS.  The commands all begin with this root.

In [ ]:
configCommandURLRoot = dataLakePlatformURL + "/servers/" + mdrServerName + "/open-metadata/access-services/discovery-engine/users/" + adminUserId

The first configuration call is to create the discovery engine.

In [ ]:
createDiscoveryEngineURL = configCommandURLRoot + '/discovery-engines'
print (createDiscoveryEngineURL)

jsonHeader = {'content-type':'application/json'}
body = {
	"class" : "NewDiscoveryEngineRequestBody",
	"qualifiedName" : "data-lake-discovery-engine",
	"displayName" : "Data Lake Discovery Engine",
	"description" : "Discovery engine used for onboarding assets."
}

response=requests.post(createDiscoveryEngineURL, json=body, headers=jsonHeader)

response.json()


In [ ]:
discoveryEngineGUID=response.json().get('guid')

print (" ")
print ("The guid for the discovery engine is: " + discoveryEngineGUID)
print (" ")

In [ ]:
createDiscoveryServiceURL = configCommandURLRoot + '/discovery-services'

print (createDiscoveryServiceURL)

jsonHeader = {'content-type':'application/json'}
body = {
	"class" : "NewDiscoveryServiceRequestBody",
	"qualifiedName" : "csv-asset-discovery-service",
	"displayName" : "CSV Asset Discovery Service",
	"description" : "Discovers columns for CSV Files.",
    "connection" : {
        "class": "Connection",
        "type": {
            "class": "ElementType",
            "elementTypeId": "114e9f8f-5ff3-4c32-bd37-a7eb42712253",
            "elementTypeName": "Connection",
            "elementTypeVersion": 1,
            "elementTypeDescription": "A set of properties to identify and configure a connector instance.",
            "elementOrigin": "CONFIGURATION"
        },
        "qualifiedName": "csv-asset-discovery-service-implementation",
        "displayName": "CSV Discovery Service Implementation Connection",
        "description": "Connection to discovery service to analyse a CSV File.",
        "connectorType": {
            "class": "ConnectorType",
            "type": {
                "class": "ElementType",
                "elementTypeId": "954421eb-33a6-462d-a8ca-b5709a1bd0d4",
                "elementTypeName": "ConnectorType",
                "elementTypeVersion": 1,
                "elementTypeDescription": "A set of properties describing a type of connector.",
                "elementOrigin": "LOCAL_COHORT"
            },
            "guid": "1111f73d-e343-abcd-82cb-3918fed81da6",
            "qualifiedName": "CSVDiscoveryServiceProvider",
            "displayName": "CSV File Discovery Service Provider Implementation",
            "description": "This connector explores the content of a CSV File.",
            "connectorProviderClassName": "org.odpi.openmetadata.accessservices.discoveryengine.samples.discoveryservices.CSVDiscoveryServiceProvider"
        }
    }
}

response=requests.post(createDiscoveryServiceURL, json=body, headers=jsonHeader)

response.json()

In [ ]:
discoveryServiceGUID=response.json().get('guid')

print (" ")
print ("The guid for the discovery service is: " + discoveryServiceGUID)
print (" ")

Finally the discovery service is registered with the discovery engine.

In [ ]:
registerDiscoveryServiceURL = configCommandURLRoot + '/discovery-engines/' + discoveryEngineGUID + '/discovery-services'

print (registerDiscoveryServiceURL)

jsonHeader = {'content-type':'application/json'}
body = {
	"class" : "DiscoveryServiceRegistrationRequestBody",
	"discoveryServiceGUID" : discoveryServiceGUID,
	"assetTypes" : [ "small-csv" ]
}

response=requests.post(registerDiscoveryServiceURL, json=body, headers=jsonHeader)

response.json()

## Exercise 2 - Configuring the Discovery Server

The discovery server is to be called `discoDL01`.

In [ ]:
organizationName = "Coco Pharmaceuticals"
serverType       = "Open Discovery Server"
discoServerName  = "discoDL01"

The code below sets up the basic properties of a governance server.

In [ ]:
adminPlatformURL = dataLakePlatformURL
adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'

print (" ")
print ("Configuring the platform that the server will run on ...")
url = adminCommandURLRoot + discoServerName + '/server-url-root?url=' + dataLakePlatformURL
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

maxPageSize = '100'

print (" ")
print ("Configuring the maximum page size ...")
url = adminCommandURLRoot + discoServerName + '/max-page-size?limit=' + maxPageSize
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the server's type ...")
url = adminCommandURLRoot + discoServerName + '/server-type?typeName=' + serverType
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the server's owning organization ...")
url = adminCommandURLRoot + discoServerName + '/organization-name?name=' + organizationName
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

discoServerUserId   = "discoDL01npa"
discoServerPassword = "discoDL01passw0rd"

print (" ")
print ("Configuring the server's userId ...")
url = adminCommandURLRoot + discoServerName + '/server-user-id?id=' + discoServerUserId
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the server's password (optional) ...")
url = adminCommandURLRoot + discoServerName + '/server-user-password?password=' + discoServerPassword
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the default audit log ...")
url = adminCommandURLRoot + discoServerName + '/audit-log-destinations/default'
print ("POST " + url)
response=requests.post(url)
print ("Response: ")
print (response.json())

print (" ")
print ("Configuring the server's security connector ...")

url = adminCommandURLRoot + discoServerName + '/security/connection'
print ("POST " + url)

header={'content-type':'application/json'}
body = {
    "class": "Connection",
    "type": {
        "class": "ElementType",
        "elementTypeId": "114e9f8f-5ff3-4c32-bd37-a7eb42712253",
        "elementTypeName": "Connection",
        "elementTypeVersion": 1,
        "elementTypeDescription": "A set of properties to identify and configure a connector instance.",
        "elementOrigin": "CONFIGURATION"
    },
    "guid": "1213abc7-2b13-4c4e-b840-97c4282f7416",
    "qualifiedName": "CocoPharmaceuticalsMetadataServerSecurityConnector",
    "displayName": "Metadata Server Security Connector",
    "description": "Connector to enforce authorization rules for accessing and updating metadata.",
    "connectorType": {
        "class": "ConnectorType",
        "type": {
            "class": "ElementType",
            "elementTypeId": "954421eb-33a6-462d-a8ca-b5709a1bd0d4",
            "elementTypeName": "ConnectorType",
            "elementTypeVersion": 1,
            "elementTypeDescription": "A set of properties describing a type of connector.",
            "elementOrigin": "LOCAL_COHORT"
        },
        "guid": "1851f73d-e343-abcd-82cb-3918fed81da6",
        "qualifiedName": "CocoPharmaServerSecurityConnectorType",
        "displayName": "Coco Pharmaceuticals Server Security Connector Implementation",
        "description": "This connector ensures only valid and authorized people can access the metadata.",
        "connectorProviderClassName": "org.odpi.openmetadata.metadatasecurity.samples.CocoPharmaServerSecurityProvider"
    }
}

response=requests.post(url, json=body, headers=header)

print ("Response: ")
print (response.json())



----
Next the discovery engine services need to be enabled.

In [ ]:

print (" ")
print ("Configuring the access service URL and Server Name  ...")

url = adminCommandURLRoot + discoServerName + '/discovery-server/access-service-root-url?accessServiceRootURL=' + dataLakePlatformURL
print ("POST " + url)
    
response=requests.post(url)

print ("Response: ")
print (response.json())

url = adminCommandURLRoot + discoServerName + '/discovery-server/access-service-server-name?accessServiceServerName=' + mdrServerName
print ("POST " + url)
  
response=requests.post(url)

print ("Response: ")
print (response.json())


print (" ")
print ("Configuring the server's discovery engine ...")

url = adminCommandURLRoot + discoServerName + '/discovery-server/set-discovery-engines'
print ("POST " + url)

header={'content-type':'application/json'}
body = [ discoveryEngineGUID ]
    
response=requests.post(url, json=body, headers=header)

print ("Response: ")
print (response.json())

## Exercise 3 - Discovering Assets

The final exercise is to run metadata discovery on a new asset.

In [ ]:
petersUserId = "peterprofile"
serverAssetOwnerURL = dataLakePlatformURL + '/servers/' + mdrServerName + '/open-metadata/access-services/asset-owner/users/' + petersUserId 

createAssetURL = serverAssetOwnerURL + '/assets/data-files/csv'
print (createAssetURL)

jsonHeader = {'content-type':'application/json'}
body = {
	"class" : "NewCSVFileAssetRequestBody",
	"displayName" : "Week 1: Drop Foot Clinical Trial Measurements",
	"description" : "One week's data covering foot angle, hip displacement and mobility measurements.",
	"fullPath" : "file://secured/research/clinical-trials/drop-foot/DropFootMeasurementsWeek1.csv"
}

response=requests.post(createAssetURL, json=body, headers=jsonHeader)

response.json()

asset1guid=response.json().get('guids')[0]

print (" ")
print ("The guid for asset 1 is: " + asset1guid)
print (" ")

print ("*THIS LAB IS UNDER CONSTRUCTION*")
discoveryCommandRootURL = dataLakePlatformURL + '/servers/' + discoServerName + '/open-metadata/discovery-server/users/' + petersUserId + '/discovery-engine/' + discoveryEngineGUID

assetType = "small-csv"
discoverAssetURL = discoveryCommandRootURL + '/asset-types/' + assetType + '/assets/' + asset1guid

#response=requests.post(discoverAssetURL)

#print ("Returns:")
#print (response.json())
#print (" ")
